In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pandas as pd # for groupbys, etc
from firebase import firebase
from datetime import datetime
import helper

In [39]:
#  import text from cloud storage
# bucket_name = "dove-hawk-data"
# source_blob_name = "data/corpus"
# destination_file_name = "corpus.txt"
# helper.download_blob(bucket_name, source_blob_name, destination_file_name)

text_df = pd.read_csv("corpus.csv", names=['date', 'text']).sort_values('date')


In [40]:
bucket_name = "dove-hawk-data"
source_blob_name = "data/new_texts"
destination_file_name = "new.txt"
helper.download_blob(bucket_name, source_blob_name, destination_file_name)

In [41]:
dates = ['2017-02-22', '2017-02-17', '2017-02-27', '2017-02-22', '2017-02-21', '2017-02-20', '2017-02-16', '2017-02-15']
texts = ["Clues awaited on ‘reverse Yankee’ comeback When Ford borrowed a €750m bond in euros at the start of the year, the US carmaker was following a path that has become", "Manufacturing exceptionalism In the debate about the importance or not of protecting manufacturing in rich countries, I often encounter an important", "Pound under pressure on new Brexit worries", "UK growth revised up; Unilever vows to unlock value as it happened", "Bank of England policymaker: We probably won't forecast the next crisis", "Greece and eurozone agree to resume bailout talks; Unilever shares fall after Kraft walks", "World stocks hit record highs as Dow keeps rising", "Deregulation of Banks Would See Only Losers-EU's Dombrovskis Rolling back international banking regulations would put the achievements made since the financial crisis into question and could trigger a dangerous race between countries to ease rules, a senior European Union official warned on Tuesday. Rolling back international banking regulations would put the achievements made since the financial crisis into question and could trigger a dangerous race between countries to ease rules, a senior European Union official warned on Tuesday"]

In [43]:
new_df = pd.DataFrame(list(zip(dates, texts)), columns=['date', 'text'])

In [44]:
new_df.head()

,date,text
0,2017-02-22,Clues awaited on ‘reverse Yankee’ comeback Whe...
1,2017-02-17,Manufacturing exceptionalism In the debate abo...
2,2017-02-27,Pound under pressure on new Brexit worries
3,2017-02-22,UK growth revised up; Unilever vows to unlock ...
4,2017-02-21,Bank of England policymaker: We probably won't...


In [46]:
text_df = text_df[1:]

In [53]:
text_df.head()

,date,text
1.0,2011-02-10,European Central Bank Candidate Said to Be Wav...
2.0,2011-02-12,Weber Steps Down as Head of Bundesbank FRANKFU...
3.0,2011-02-12,Head of German Central Bank Is Stepping Down A...
4.0,2011-02-14,Politics Will Determine New E.C.B. Head The ma...
5.0,2011-02-16,German Aide Thought Likely to Become Head of B...


In [73]:
concat_df = pd.concat([text_df, new_df], axis=0)
concat_df.dropna(inplace=True)

In [74]:
concat_df.head()

,date,text
1.0,2011-02-10,European Central Bank Candidate Said to Be Wav...
2.0,2011-02-12,Weber Steps Down as Head of Bundesbank FRANKFU...
3.0,2011-02-12,Head of German Central Bank Is Stepping Down A...
4.0,2011-02-14,Politics Will Determine New E.C.B. Head The ma...
5.0,2011-02-16,German Aide Thought Likely to Become Head of B...


In [50]:
text_df.shape

(5724, 2)

In [77]:
from datetime import datetime
timestamps = [datetime.strptime(date, "%Y-%m-%d") for date in concat_df.date]
weeks = [datetime.date(t).isocalendar()[:2] for t in timestamps]

weeks = []
for t in timestamps:
    w = datetime.date(t).isocalendar()[:2]
    year = str(list(w)[0])
    week = str(list(w)[1])
    if len(week) == 1:
        week = '0'+week
    
    date = year+week
    weeks.append(date)

In [78]:
concat_df['week'] = weeks

In [79]:
week_groups = concat_df.groupby("week")
week_sorted = list(set(concat_df.week))
week_sorted = sorted(week_sorted)

In [170]:
corpus = []
for week in week_sorted:
    group = week_groups.get_group(week)
    doc = ''.join(group["text"])
    corpus.append(doc)

In [171]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform(corpus)
feature_names = tf.get_feature_names() 

In [168]:
week_groups.get_group('201709')['text']

2.0    Pound under pressure on new Brexit worries
Name: text, dtype: object

In [177]:
data = {}
doc_id = 0
for doc in tfidf_matrix.todense():
    word_id = 0
    word_scores = []

    for score in doc.tolist()[0]:
        if score > 0:
            word = feature_names[word_id]
            tup = (score*1000, word.encode("utf-8"))
            word_scores.append(tup)
        word_id +=1
    doc_id +=1

    word_scores.sort()

#     i = 0    
    for score, word in word_scores[-50:]:
        if week_sorted[doc_id] in data.keys():
            data[week_sorted[doc_id]]['words'].append({"value":str(word).strip("b").strip("'"),"weight":int(score)})
#             i+=1 
        else:
            data[week_sorted[doc_id]] = {'words':[{"value":str(word).strip("b").strip("'"),"weight":int(score)}]}



IndexError: list index out of range

In [175]:
data['201708']

{'words': [{'value': 'meet expectationsderegulation', 'weight': 51},
  {'value': 'protecting manufacturing', 'weight': 51},
  {'value': 'reports sluggish', 'weight': 51},
  {'value': 'risingderegulation', 'weight': 51},
  {'value': 'risingderegulation banks', 'weight': 51},
  {'value': 'tuesday manufacturing', 'weight': 51},
  {'value': 'uk economyeurozone', 'weight': 51},
  {'value': 'banking', 'weight': 60},
  {'value': 'union', 'weight': 60},
  {'value': 'international', 'weight': 62},
  {'value': 'tuesday', 'weight': 64},
  {'value': 'warned', 'weight': 67},
  {'value': 'countries', 'weight': 71},
  {'value': 'official', 'weight': 71},
  {'value': 'european union', 'weight': 76},
  {'value': 'ease', 'weight': 76},
  {'value': 'financial crisis', 'weight': 77},
  {'value': 'losers', 'weight': 78},
  {'value': 'rules', 'weight': 80},
  {'value': 'question', 'weight': 81},
  {'value': 'senior', 'weight': 83},
  {'value': 'dombrovskis', 'weight': 90},
  {'value': 'banks losers', 'weigh

In [176]:
# word_scores = []
# current_week_tfidf = tfidf_matrix[-1:].todense()
# word_id = 0

# for score in current_week_tfidf.tolist()[0]:
#     if score > 0:
#         word = feature_names[word_id]
#         tup = (score, word.encode("utf-8"))
#         word_scores.append(tup)
#     word_id += 1
    
# word_scores.sort()

# i = 0    
# for score, word in word_scores[-50:]:
#     data[i] = {"value":str(word).strip("b").strip("'"),"weight":1000*int(score)}
#     i+=1

        

In [178]:
# firebase = firebase.FirebaseApplication('https://dl-dove-hawk.firebaseio.com/', None)
firebase.put("/data/wordclouds/tfidf", data=data, name='weeks')

{'201107': {'words': [{'value': 'confusion', 'weight': 77},
   {'value': 'central bank', 'weight': 77},
   {'value': 'jean', 'weight': 78},
   {'value': 'resigned', 'weight': 79},
   {'value': 'regarding', 'weight': 81},
   {'value': 'claude', 'weight': 81},
   {'value': 'jean claude', 'weight': 81},
   {'value': 'bundesbank said', 'weight': 83},
   {'value': 'quandary', 'weight': 83},
   {'value': 'expected german', 'weight': 91},
   {'value': 'head', 'weight': 93},
   {'value': 'bank year', 'weight': 95},
   {'value': 'claude trichet', 'weight': 99},
   {'value': 'bank later', 'weight': 100},
   {'value': 'bank resigned', 'weight': 100},
   {'value': 'public service', 'weight': 100},
   {'value': 'runner head', 'weight': 100},
   {'value': 'trichet president', 'weight': 100},
   {'value': 'head european', 'weight': 101},
   {'value': 'candidate', 'weight': 101},
   {'value': 'announcement regarding', 'weight': 108},
   {'value': 'confusion creating', 'weight': 108},
   {'value': 'con